In [1]:
with open('../../my_key.txt') as f:
    key_list = f.readlines()

key_list = [key.strip() for key in key_list]

keys = {}
for key in key_list:
    key = key.split(':')
    if (key != ['']):
        keys[key[0]] = key[1]

In [2]:
import os

os.environ['OPENAI_API_KEY'] = keys['OPENAI_API_KEY']

## Tiktoken
- Use to count the number of token

In [3]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [26]:
text_to_token = ' [[url4]] '
tokens_list = encoding.encode(text_to_token)
print(tokens_list)

[4416, 1103, 19, 5163, 220]


In [16]:
encoding.decode([12805])

'Once'

In [ ]:
len(tokens_list)

## OpenAI Original Call
- prompt 1765 + completion 226 = 
    - 'gpt-3.5-turbo': 18s+14s+15s /3 = 16s
    - 'text-davinci-003': above 22s

In [12]:
import openai
import time

openai.organization = keys['OPENAI_ORGANIZATION']
openai.api_key = os.getenv('OPENAI_API_KEY')
# openai.Model.list()

In [13]:
text1_content = "A tiger does not lose sleep over the opinion of sheep."
text2_content = "The tiger's strength is born from the struggle."

### gpt-3.5-turbo
- Without Modification

In [ ]:
sys_prompt = """You are a story teller. Make a story from two sentences."""

text = text1_content + '\n' + text2_content

usr_prompt = f"""
Sentences: {text}
"""

start = time.time()
tur_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content": f'{sys_prompt}'},
        {"role": 'user', 'content': f'{usr_prompt}'},
    ],
    temperature = 0,
)
stop = time.time()

In [ ]:
print(str(tur_response) + '\n' + str(stop - start) + 's')

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The article compares two popular language models, GPT-3 and BERT, and discusses their differences and similarities, exploring their capabilities and the tools that use them. GPT-3 is an autoregressive language model developed by OpenAI, while BERT is a bidirectional transformer model developed by Google AI. The main difference between the two models lies in their architecture and training dataset size, with GPT-3 being better suited for tasks such as summarization or translation, while BERT is more beneficial for sentiment analysis or natural language understanding (NLU). However, both models have proven themselves valuable tools for performing various NLP tasks with varying degrees of accuracy.",
        "role": "assistant"
      }
    }
  ],
  "created": 1685508068,
  "id": "chatcmpl-7M7vwE8NQeN2sDGdZBmt35eFvgzdK",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completio

In [ ]:
tur_response['choices'][0]['message']['content']

'The article compares two popular language models, GPT-3 and BERT, and discusses their differences and similarities, exploring their capabilities and the tools that use them. GPT-3 is an autoregressive language model developed by OpenAI, while BERT is a bidirectional transformer model developed by Google AI. The main difference between the two models lies in their architecture and training dataset size, with GPT-3 being better suited for tasks such as summarization or translation, while BERT is more beneficial for sentiment analysis or natural language understanding (NLU). However, both models have proven themselves valuable tools for performing various NLP tasks with varying degrees of accuracy.'

### gpt-3.5-turbo
- With Streaming
    - use 'stream = True'
    - automatically call 'await'

In [18]:
sys_prompt = """You are a story teller. Make a story from two sentences."""

text = text1_content + '\n' + text2_content

usr_prompt = f"""
Sentences: {text}
"""

tur_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content": f'{sys_prompt}'},
        {"role": 'user', 'content': f'{usr_prompt}'},
    ],
    temperature = 0,
    stream = True
)

i = 0
NUM_TOKEN_OUT = 3
text_chunk = ''
for chunk in tur_response:
    try:
        text_chunk += chunk['choices'][0]['delta']['content']
        if (i % NUM_TOKEN_OUT == 0):
            print(text_chunk)
            text_chunk = ''
        i += 1
    except:
        pass

Once
 upon a time
, in a
 dense forest,
k lived a
 majestic tiger.
 He was feared
 by all the
 animals in the
 forest, but
 he never cared
 about their opinions
. He knew
 that he was
 strong and powerful
, and his
 strength was born
 from the struggles
 he faced in
 his life.


One day,
 a group of
 sheep came to
 the tiger and
 started mocking him
 for his stripes
 and his fer
ocious nature.
 The tiger just
 smiled and said
, "A
 tiger does not
 lose sleep over
 the opinion of
 sheep." The
 sheep were surprised
 by the tiger
's response and
 realized that they
 were no match
 for his strength
 and courage.


From that day
 on, the
 tiger became a
 legend in the
 forest, and
 all the animals
 respected him for
 his bravery and
 wisdom. He
 continued to face
 struggles and challenges
, but he
 never lost his
 confidence and always
 emerged victorious.


The tiger's
 story became an
 inspiration for all
 the animals in
 the forest,
 and they learned
 that true strength
 comes from withi

#### Test

In [ ]:
sys_prompt = """You are a story teller. Make a story from two sentences."""

text = text1_content + '\n' + text2_content

usr_prompt = f"""
Sentences: {text}
"""

tur_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content": f'{sys_prompt}'},
        {"role": 'user', 'content': f'{usr_prompt}'},
    ],
    temperature = 0,
    stream = True
)

i = 0
j = 0
NUM_TOKEN_OUT = 3
text_chunk = ''
for chunk in tur_response:
    try:
        if (chunk['choices'][0]['delta']['content'] == ' there'):
            j += 1
        text_chunk += chunk['choices'][0]['delta']['content']
        if (i % NUM_TOKEN_OUT == 0):
            print(text_chunk)
            text_chunk = ''
        i += 1
    except:
        pass

### text-davinci-003

In [ ]:
text = text1_content + '\n' + text2_content

usr_prompt = f"""You are a story teller. Make a story from two sentences.
Sentences: {text}
Story:
"""

start = time.time()
da_response = openai.Completion.create(
    engine = 'text-davinci-003',
    prompt = usr_prompt,
    max_tokens = 300
)
stop = time.time()

In [ ]:
da_response

<OpenAIObject text_completion id=cmpl-7M6v1z4mPhssrfNtkAWmzPXFAdDk4 at 0x7f85c38ca150> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "GPT-3 and BERT are two well-known language models used in natural language processing (NLP) applications. GPT-3 is an autoregressive model trained on 45TB of data while BERT is bidirectional and trained on 3TB of data. They have significant differences in architecture and data sizes, but both are Transformer-based and can be used for similar tasks such as question answering, summarization, and translation. GPT-3 typically outperforms BERT in tasks that require more data, while BERT is better for sentiment analysis and natural language understanding. GPT-2 models can be used to generate data from prompts, while BERT can be used for zero- and few-shot learning with the PET method. [from url1 and url2]"
    }
  ],
  "created": 1685504167,
  "id": "cmpl-7M6v1z4mPhssrfNtkAWmzPXFAdDk4",
  "m

In [ ]:
da_response['choices'][0]['text']

'GPT-3 and BERT are two well-known language models used in natural language processing (NLP) applications. GPT-3 is an autoregressive model trained on 45TB of data while BERT is bidirectional and trained on 3TB of data. They have significant differences in architecture and data sizes, but both are Transformer-based and can be used for similar tasks such as question answering, summarization, and translation. GPT-3 typically outperforms BERT in tasks that require more data, while BERT is better for sentiment analysis and natural language understanding. GPT-2 models can be used to generate data from prompts, while BERT can be used for zero- and few-shot learning with the PET method. [from url1 and url2]'